In [ ]:
import json
from datetime import datetime, timedelta

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
#fname = 'bigpanda.hc_20324124.json'
#fname = 'bigpanda.hc20323822.tid1313.json'
#fname = 'bigpanda.hc20324124.tid1337.json'
#fname = 'bigpanda.hc20327391.tid1381.json'
#fname = 'bigpanda.hc20327453.tid1382.json'
#fname = 'bigpanda.hc20327553.tid1381.json'
#fname = 'bigpanda.hc20327629.tid1381.json'
#fname = 'bigpanda.hc20327687.tid1381.json'
#fname = 'bigpanda.hc20327751.tid1381.json'
fname = 'bigpanda.hc20327824.tid1381.json'
#fname = 'bigpanda.hc20327828.tid1339.json'
jd = json.load(open(fname))

In [ ]:
jd.keys()


In [ ]:
len(jd['jobs'])

In [ ]:
jd['jobs'][0]

In [ ]:

df = pd.DataFrame(jd['jobs'])
len(df)

In [ ]:
# cleanup dataset and add some further parameters
df = df[df.jobstatus=='finished'] # only finished jobs
print(len(df))

df['readfrac'] = df.totrchar*1024/df.inputfilebytes
# calculate input rate in MB/s
df['readrate'] = df.totrchar/1024/df.durationsec
df['evtrate'] = df.nevents/df.durationsec
# convert start/end time to date
df['starttime'] = pd.to_datetime(df['starttime'])
df['endtime'] = pd.to_datetime(df['endtime'])
df['cputype']=[x[2:16] for x in df.cpuconsumptionunit]
# work load run-time from pilottiming list
df['wlruntime'] = [int(x.split('|')[2]) for x in df.pilottiming]
df['wlreadrate'] = df.totrchar/1024/df.wlruntime
df['wlcpueff'] = df.cpuconsumptiontime/df.wlruntime

# only entries later than ...
#df = df[df.starttime>datetime.fromisoformat('2026-01-22 16:00:00')]

# calculate avg time of job in mins since start of test 
st0 = df.starttime.min().floor('min')
# mean time of runing job 
avt = df.starttime+(df.endtime-df.starttime)/2
dt  = [ (st-st0).total_seconds() / 60 for st in  avt]
df['avg_minute'] = dt
df['start_minute'] = [ int((st-st0).total_seconds()) / 60 for st in  df.starttime]



print(len(df))

In [ ]:


# investigate pilottiming

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 8),constrained_layout=True)

for ax,i in zip(axes.flatten(),range(6)):
    v = [int(x.split('|')[i]) for x in df.pilottiming]
    ax.hist(v,bins=30)
    ax.set_xlabel('pilottiming '+str(i))

fig.savefig('pilottiming.png')

In [ ]:
df.plot.scatter('wlreadrate','raterchar');

In [ ]:
df.plot.scatter('inputfilebytes','totrchar');

In [ ]:
print(f'total GB read {df.totrchar.sum()/1e6:.3f}\ntotal GB filesize {df.inputfilebytes.sum()/1e9:.3f}')

In [ ]:
# some basic dists

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 8),constrained_layout=True)
pcols = ['readrate','cpuefficiency','wlruntime','readfrac','evtrate']

for x,p in zip(axes.flatten(),pcols):
    x.hist(df[p],bins=30)
    x.set_xlabel(p)



In [ ]:
#df.hist('readrate',by='cputype',sharex=True)
# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 6), constrained_layout=True)

# Using seaborn's displot for overlayed histograms
axes = axes.flatten()
sns.histplot(data=df, ax=axes[0], x='readrate', hue='cputype', bins=30)
sns.histplot(data=df, ax=axes[1], x='cpuefficiency', hue='cputype', bins=30)
sns.histplot(data=df, ax=axes[2], x='wlreadrate', hue='cputype', bins=30)
sns.histplot(data=df, ax=axes[3], x='wlcpueff', hue='cputype', bins=30)


In [ ]:
#df.plot.scatter('readrate','cpuefficiency');
#df.plot.scatter('readrate','wlcpueff');
df.plot.scatter('wlcpueff','cpuefficiency');

In [ ]:
# timeline of jobs and IO rate

st = df.starttime.min().floor('min')
et = df.endtime.max().ceil('min')
minutes_diff = (et-st).total_seconds() / 60
st,et,minutes_diff

In [ ]:
# calculate sum of running jobs and sum of transfer-rate vs time

nbins = int(minutes_diff+1)
bins = np.arange(nbins+1)
counts = np.zeros(nbins)
trate = np.zeros(nbins)
ct = st
for i in range(nbins):
    counts[i] = df[(df.starttime<ct) &  (df.endtime>ct)].readrate.count()
    trate[i]  = df[(df.starttime<ct) &  (df.endtime>ct)].readrate.sum()
    ct += timedelta(minutes=1)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
ax = axes[0]
ax.hist(bins[:-1],bins,weights=counts)
ax.set_xlabel('time (mins)')
ax.set_ylabel('#-jobs');

ax = axes[1]
ax.hist(bins[:-1],bins,weights=trate)
ax.set_xlabel('time (mins)')
ax.set_ylabel('total rate (MB/s)');
fig.suptitle('HC stress test transfers from panda job par');
#fig.savefig('hc_stress_es_jobpar.png')

In [ ]:
plt.plot(df.avg_minute, df.readrate, '.')


In [ ]:
# plot cpu-eff & transfer rate vs time
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
ax = axes[0]
ax.plot(df.avg_minute, df.readrate,'.')
ax.set_xlabel('time ')
ax.set_ylabel('read-rate (MB/s)');

ax = axes[1]
ax.plot(df.avg_minute, df.cpuefficiency,'.')
#ax.plot(df.avg_minute, df.wlcpueff,'.')
ax.set_xlabel('time ')
ax.set_ylabel('CPU-eff');
#fig.savefig('hc_stress_es_jobpar.png')

In [ ]:
#njobs = [counts[int(x)] for x in df.avg_minute]

# avg number of active jobs during job execution
df['njobs'] = [sum(counts[int(s):int(s+t)])/t for s,t in zip(df.start_minute,df.durationmin)]


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
ax = axes[0]
ax.plot(df.njobs, df.readrate, '.')
ax.set_xlabel('# active jobs')
ax.set_ylabel('read-rate (MB/s)');

ax = axes[1]
ax.plot(df.njobs, df.cpuefficiency, '.')
#ax.plot(njobs, df.wlcpueff,'.')
ax.set_xlabel('# active jobs')
ax.set_ylabel('CPU-eff');

#plt.plot(njobs, df.readrate, '.')
#plt.plot(njobs, df.cpuefficiency, '.')



In [ ]:
bmax=250
nb = 41
bwid = bmax/(nb-1)
bins = np.linspace(0, bmax, nb)
binc = np.digitize(df.njobs, bins)

ax = sns.boxplot(x=binc, y=df.cpuefficiency);

# hack to fix x-axis labels
ax.set_xticks(((bins-bwid)/bwid)[::5])
ax.set_xticklabels(bins[::5]);
plt.grid()

In [ ]:
#df = df[df.cputype=='AMD EPYC 9654 ']

In [ ]:
# reduce df size
dfr = df.sample(frac=0.3, random_state=0)
len(dfr)

In [ ]:
# output df as pickle
fout = fname[:fname.rfind('.')]+'.pkl'
#df.to_pickle(fout)
dfr.to_pickle(fout)